# Calculation of All Field Components (E, H, S)

This example shows how to calculate all six field components (Hx, Hy, Hz·j) and (Ex, Ey, Ez·j) for a silicon channel waveguide with oxide cladding layer, and how to find the Poynting vector S.  The magnetic field components are computed at the grid vertices (nx+1,ny+1), while the electric field components are computed at the cell centers (nx,ny).

This example demonstrates that the transverse electric fields (Ex, Ey) are singular at sharp dielectric corners, as explained in 

**Why are accurate computations of mode fields in rectangular dielectric waveguides difficult?**<br>
A. S. Sudbo<br>
*J. of Lightwave Technol*, **10**(4) 418 (1992).  
<https://doi.org/10.1109/50.134193>

In [ ]:
# Enable automatic reloading of modules (IPython only)
%load_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
n1 = 1.46           # SiO2 lower cladding
n2 = 3.50           # Silicon core
n3 = 1.46           # Air upper cladding

# Vertical dimensions:
h1 = 500            # lower cladding (nm), will be stretched
h2 = 200            # silicon core (nm)
h3 = 500            # upper cladding (nm), will be stretched

# Horizontal dimensions:
w = 400             # full width (nm)
side = 500          # Space on side, will be stretched

# Grid size:
dx = 2.5            # grid size (x)
dy = dx             # grid size (y)

# Build waveguide mesh (eps)
x,y,xc,yc,nx,ny,eps,edges = mode.waveguidemesh([n1,n2,n3],[h1,h2,h3],h2,w/2,side,dx,dy,return_edges=True)

In [ ]:
# Stretch cladding layers (NORTH, SOUTH, EAST) by 1.5X
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[200,200,200,0],[1.5,1.5,1.5,1])

In [ ]:
wavelength = 1550   # vacuum wavelength (nm)
nmodes = 1          # number of modes to compute

# First, consider the fundamental TE mode:
neff, Ex, Ey, Ezj, Hx, Hy, Hzj = mode.wgmodes(wavelength,n2,nmodes,dx,dy,eps=eps,boundary='000A')
print(f"neff = {neff:.6f}")

In [ ]:
# E fields are normalized using η0 = sqrt(µ0/ε0) relative to H.  Here we rescale fields 
# so that they are instead normalized with a factor of η2 = sqrt(µ0/ε2), where ε2 = core 
# material permittivity

Ex *= n2
Ey *= n2
Ezj *= n2

# Calculate Poynting vector S (assume Sx = Sy = 0, so only find Sz)
_, _, Sz = mode.poynting(Ex, Ey, Ezj, Hx, Hy, Hzj)
core = np.isclose(eps, n2**2)
Sz /= Sz[core].max()   # normalize to maximum of Sz within core

In [ ]:
# Plot all 6 field components, with common axes
fig, axes = plt.subplots(2,3,sharex=True,sharey=True,figsize=(17,20))
fig.subplots_adjust(wspace=0, hspace=0)

for ax, field, title in zip(axes[0,:], [Ex, Ey, Ezj], ["Ex", "Ey", "Ez*j"]):
    ax.pcolormesh(xc,yc,abs(field),cmap='jet',vmax=1)
    ax.set_aspect('equal')
    ax.text(
        0.95, 0.95,                 # (x, y) in axes fraction coordinates
        title,                      # text string
        transform=ax.transAxes,     # interpret as axes coordinates
        color="white",              # text color
        weight="bold",              # bold font
        fontsize=14,                # font size
        ha="right", va="top"        # horizontal/vertical alignment
    )
    l = LineCollection(edges, colors='black', linewidths=1)
    ax.add_collection(l)

for ax, field, title in zip(axes[1,:], [Hx, Hy, Hzj], ["Hx", "Hy", "Hz*j"]):
    ax.pcolormesh(x,y,abs(field),cmap='jet',vmax=1)
    ax.set_aspect('equal')
    ax.text(
        0.95, 0.95,                 # (x, y) in axes fraction coordinates
        title,                      # text string
        transform=ax.transAxes,     # interpret as axes coordinates
        color="white",              # text color
        weight="bold",              # bold font
        fontsize=14,                # font size
        ha="right", va="top"        # horizontal/vertical alignment
    )
    l = LineCollection(edges, colors='black', linewidths=1)
    ax.add_collection(l)

plt.show()

In [ ]:
# Plot Poynting vector (Sz)
fig, ax = plt.subplots()

ax.pcolormesh(xc,yc,Sz,cmap='jet',vmin=0,vmax=1)
ax.set_aspect('equal')
ax.axis([min(x), max(x), min(y), max(y)])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Sz')
l = LineCollection(edges, colors='black', linewidths=1)
ax.add_collection(l)

plt.show()